# <span style='background:yellow'> Pacer's BikeShare Data Cleanup and Visualization </span>
##### The purpose of the following code is to visualize and clean bikeshare data placed in the `kiosk_data` folder. The graphs and visualizations generated are placed in the '`month_histogram`' and '`other_graphs`' folders. There is a `gis_compatible.csv` file generated by this program to more-easily visualize bike station usage within GIS software.
   ---
*The code that cleans the data is hidden by default**

## Step 1: Import Needed Libraries & Files
---
First, we will import the needed libraries.

In [ ]:
import pandas as pd # for manipulating .csv files as dataframes
import glob #for importing files
import os # for navigating folders
import matplotlib.pyplot as plt # for graphs
import seaborn as sns # another graphing library
import calendar
from datetime import datetime

Next, we will import the needed files.
- After the needed libraries have been imported, the data needs to be loaded into python. This will be done by loading all the .csv files into a dataframe. A problem that arises in doing this is that there are potentially many files being imported. We don't know the names of these files, or how many there will be. To remedy this problem, we will use both the `glob` and `os` libraries to:
> 1) Get the present working directory (folder) with `os.getcwd` and save this path to `path`.
> 2) Save the names of all the .csv files in the kiosk_data folder as a list of strings `all_files`. This is accomplished through the `glob` library
> 3) Iterate through every file in the newly created list, save each file as a dataframe, and then merge all the dataframes together.

In [ ]:
# os.getcwd() gets the present working directory (pwd) AKA *current folder*
all_files = glob.glob(os.path.join(os.getcwd(), "kiosk_data", "*.csv"))

df_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0,encoding='latin-1')
    df_list.append(df)

df = pd.concat(df_list, axis=0, ignore_index=True)

The station locations must be loaded too.

In [ ]:
locations  = pd.read_csv(os.path.join(os.getcwd(), 'station_location.csv'))

## Step 2: Understand the Data
---


In [ ]:
df.head()

## Step 3: Clean the Data
---
- As seen in the output above, a lot of the data generated is from `"Maintenance"`, this presumably represents maintenance workers, not end users. Let's see how many of these entries there are.

How many maintenance rows are there?

In [ ]:
len(df[df.UserRole.str.contains("Maintenance") == True])

### Removing maintenance rows
- The dataframe without maintenance rows will be saved as `pacer_data`. The following snippet shows how much this reduction saved in terms of dataframe size.

In [ ]:
# removing maintenance roles.
pacer_data = df[df.UserRole.str.contains("Maintenance") == False]
print(f'Size before maintenance deletion: {df.size}\n Size after maintenance deletion: {pacer_data.size}')
print(f'Length before maintenance deletion: {len(df)}\n Length after maintenance deletion: {len(pacer_data)}')

### Dropping unnecessary columns from dataframe
- There were a lot of columns in the dataframe that won't be used. They'll be omitted to allow for faster runtime.

In [ ]:
# dropping most columns from main dataframe...
old_size = pacer_data.size
pacer_data = pacer_data.drop(['TripId','UserProgramName','UserCity','UserState','UserZip','UserCountry','MembershipType','Bike','BikeType',
                 'UserId', 'UserRole','DurationMins','AdjustedDurationMins','UsageFee','AdjustmentFlag',
                  'EstimatedCaloriesBurned','LocalProgramFlag','TripRouteCategory','TripProgramName'], axis=1)
print(f'Size before column deletion: {old_size}\n Size after column deletion: {pacer_data.size}')
print(f'Net decrease in size: {(pacer_data.size / old_size):2.2%}')

### Sorting by checkout date

In [ ]:
pacer_data = pacer_data.sort_values(by="CheckoutDateLocal")
pacer_data = pacer_data.reset_index(drop=True)
pacer_data.head()

This finds the start and end dates of the table
 (This will be useful in naming file output)

In [ ]:
start_date = pacer_data.at[pacer_data.index[0], "CheckoutDateLocal"]
end_date = pacer_data.at[pacer_data.index[-1], "CheckoutDateLocal"]
print(f'The table represents trips spanning from {start_date} to {end_date}')

In [ ]:
#parsing the start and end dates as datetime, taking the difference, then taking the absolute value.
total_days = abs((datetime.strptime(start_date, '%Y-%m-%d') - datetime.strptime(end_date, '%Y-%m-%d')).days)
total_days

Adding additional columns for latitude and longitude and populating them with a default value of zero. This creates "empty" columns that will then be filled by adding the coordinates of their respective kiosks.

In [ ]:
# d.insert("column index", "column name", "default value") 
pacer_data.insert(1, "Checkout_Latitude", 0)
pacer_data.insert(2, "Checkout_Longitude", 0)
pacer_data.insert(4, "Checkin_Latitude", 0)
pacer_data.insert(5, "Checkin_Longitude", 0)
pacer_data.head()

- The DATAFRAME and CSV populate differently. An example of this is the `Michigan St. and N White River Pkwy.` location (it has an an additional space after the final period). The following code attempts to populate the longitude and latitude columns for these "bad" locations.

In [ ]:
pacer_data['CheckoutKioskName'].replace({"Michigan St. and N White River Pkwy. ": "Michigan St. and N White River Pkwy.",
                                         "Michigan and Blackford":"Michigan St. and Blackford",
                                        "Michigan and Senate":"Michigan St. and Senate",
                                        "State Fairgrounds and Monon Trail":"State Fairgrounds at 38th St. and Monon Trail"}, inplace=True)

- This code loops through all the station names in the `station_locations.csv` folder and gets the latitude and longitude from them to add to the dataframe.

In [ ]:
for i in range(len(locations['Station Name'])):
    #checkout kiosk
    pacer_data.loc[pacer_data['CheckoutKioskName'] == locations['Station Name'][i], 'Checkout_Latitude'] = locations.Latitude[i]
    pacer_data.loc[pacer_data['CheckoutKioskName'] == locations['Station Name'][i], 'Checkout_Longitude'] = locations.Longitude[i]
    #return kiosk
    pacer_data.loc[pacer_data['ReturnKioskName'] == locations['Station Name'][i], 'Checkin_Latitude'] = locations.Latitude[i]
    pacer_data.loc[pacer_data['ReturnKioskName'] == locations['Station Name'][i], 'Checkin_Longitude'] = locations.Longitude[i]

In [ ]:
pacer_data.head()

In [ ]:
bad_locations = pacer_data.loc[pacer_data['Checkout_Latitude'] == 0, 'CheckoutKioskName'].unique()
for location in bad_locations:
    print(location)
bad_locations
# Washington and Illinois wasn't listed as a station anyways. Disregard.
# Headquarters is all maintenance.

In [ ]:
pacer_data = pacer_data[pacer_data['Checkout_Latitude'] != 0]

In [ ]:
print(pacer_data.nunique())

In [ ]:
carbon_offset = pacer_data['EstimatedCarbonOffset'].sum()
print(f'The total carbon offset is {carbon_offset/1000:,.2f} kg CO2')

### Populating New Columns

In [ ]:
print(str(len(pacer_data["CheckoutDateLocal"])) + ' ' + str(len(pacer_data["CheckoutTimeLocal"])))
pacer_data["CheckoutDateTime"] = pacer_data['CheckoutDateLocal'] + ' ' + pacer_data['CheckoutTimeLocal']
pacer_data["ReturnDateTime"] = pacer_data['ReturnDateLocal'] + ' ' + pacer_data['ReturnTimeLocal']

In [ ]:
pacer_data["CheckoutDateTime"] = pd.to_datetime(pacer_data["CheckoutDateTime"])
pacer_data["ReturnDateTime"] = pd.to_datetime(pacer_data["ReturnDateTime"])
pacer_data["CheckoutDateLocal"] = pd.to_datetime(pacer_data["CheckoutDateLocal"])

In [ ]:
#Checking to see when the weekend checkouts are
pacer_data["IsWeekend"] = pacer_data["CheckoutDateLocal"].dt.weekday >= 5
print("Total Checkouts: " + str(len(pacer_data[pacer_data["IsWeekend"] == True])))
print("Weekend Checkouts: " + str(len(pacer_data[pacer_data["IsWeekend"] == True])))
print("Weekday Checkouts: " + str(len(pacer_data[pacer_data["IsWeekend"] == False])))

# Step 4: Visualize the Data
---
## Histograms: Month and Year
- This program was made to display different times separately, namely: **weekends vs weekdays** as well as **months**, and **years**
    - The different time categories will be distinguished before setting them in a `for` loop...
        - A function will iterate through every *unique* year, followed by every *unique* month in that year.
        - This will create a histogram for each year (saved in `other_graphs`) and a histogram for each month (saved in `monthly_histograms`)

In [ ]:
# looping through years
for year in pacer_data['CheckoutDateTime'].dt.year.unique():
    print('---\n', year ,'\n---')
    year_pacer_data = pacer_data[pacer_data['CheckoutDateTime'].dt.year == year]
    
    numeric_time_series = year_pacer_data["CheckoutDateTime"].dt.hour + .01*year_pacer_data["CheckoutDateTime"].dt.minute
    year_hist = sns.histplot(data=numeric_time_series, bins=24, element="step", kde=True)
    plt.xlabel("Time of Day", fontsize=12)
    plt.title(f"Checkouts by Time of Day: {year}", fontsize=12)
    year_hist2 = year_hist.get_figure()
    year_hist2.savefig(str(os.path.join(os.getcwd(),'other_graphs', f'{year}_checkout_summary')))
    plt.clf() # This clears the figure so it doesn't overlap.
    print(str(os.path.join(os.getcwd(),'other_graphs', f'{year}_checkout_summary')))
    
    # now iterate across the months
    for month in year_pacer_data['CheckoutDateTime'].dt.month.unique():
        ##
        month_word = calendar.month_name[month]
        #create a month dataframe out of the year dataframe...
        month_pacer_data = year_pacer_data[year_pacer_data['CheckoutDateTime'].dt.month == month]
        month_pacer_data = pd.to_datetime(month_pacer_data['CheckoutDateTime'])
        numeric_time_series = month_pacer_data.dt.hour + .01*month_pacer_data.dt.minute
        ##
        #------- P L O T T I N G -------
        ##
        sns_hist = sns.histplot(data=numeric_time_series, bins=24, element="step", kde=True)
        plt.xlabel("Time of Day", fontsize=12)
        plt.title(f"Checkouts by Time of Day: {month_word} {year}", fontsize=12)
        #plt.set_xticks([], minor=True)
        shist = sns_hist.get_figure()
        month_file = f'checkout_histogram_{month}_{year}'
        shist.savefig(str(os.path.join(os.getcwd(), 'month_histograms', month_file)))
        plt.clf() # This clears the figure so it doesn't overlap.
        print(os.path.join(os.getcwd(), 'month_histograms', month_file))

## Creating the Complete Histogram
- This histogram spans the entire duration of the months provided in `kiosk_data`

In [ ]:
numeric_time_series = pacer_data["CheckoutDateTime"].dt.hour + .01*pacer_data["CheckoutDateTime"].dt.minute
sns_hist = sns.histplot(data=numeric_time_series, bins=24, element="step", kde=True)
plt.xlabel("Time of Day", fontsize=12)
plt.title(f"Checkouts by Time of Day: {start_date} to {end_date}", fontsize=12)
#plt.set_xticks([], minor=True)
plt.savefig(str(os.path.join(os.getcwd(),'other_graphs', f'complete_checkout_histogram_{start_date}_{end_date}')))

***
## Creating a Formatted `.csv` output file
- Getting Total Checkout counts

In [ ]:
summary_df = locations

In [ ]:
def count_check_return(pacer_data, summary_df):
    summary_df.insert(len(summary_df.columns), "Checkout Count", 0)
    #populate checkout counts
    checkout_counts =pacer_data['CheckoutKioskName'].value_counts()
    checkcount1 = checkout_counts.to_dict() #converts to dictionary
    summary_df['Checkout Count'] = summary_df['Station Name'].map(checkcount1) 
count_check_return(pacer_data, summary_df)

- This creates a csv file containing:
    - All the station names
    - The GPS coordinates for each station name (World Mercator 84)
    - The **Total Checkouts** for each station

In [ ]:
summary_df.to_csv('gis_compatible.csv')

## Study: Visualizing the total Carbon Offset over Time
- A good way to visualize this is through a bar graph.

In [ ]:
years = []
annual_carbon_offset = []
net_carbon_offset = []

for year in pacer_data['CheckoutDateTime'].dt.year.unique():
    year_pacer_data = pacer_data[pacer_data['CheckoutDateTime'].dt.year == year]
    #year
    years.append(year)
    #annual carbon offet
    carbon_offset = year_pacer_data['EstimatedCarbonOffset'].sum() / 1000
    annual_carbon_offset.append(carbon_offset)
    # net carbon offset
    net_carbon_offset.append(sum(annual_carbon_offset))

In [ ]:
carbon_offset_df = pd.DataFrame([years, annual_carbon_offset, net_carbon_offset]).transpose()
carbon_offset_df.columns = ['Year', 'Annual CO2 Offset', 'Net CO2 Offset']
carbon_offset_df['Year'] = carbon_offset_df['Year'].astype(int)
carbon_offset_df

In [ ]:
fig, ax1 = plt.subplots()
tidy = carbon_offset_df.melt(id_vars='Year').rename(columns=str.title)
sns.barplot(x='Year', y='Value', hue='Variable', palette="ch:s=2.2,rot=0,dark=0.4, light=0.8", data=tidy, ax=ax1)
plt.ylabel("Carbon Offset (kg)", fontsize=12)
sns.despine(fig)
plt.gcf().subplots_adjust(left=0.2)
plt.legend()
plt.title('Carbon Reduction by Year')
plt.savefig(str(os.path.join(os.getcwd(),'other_graphs','annual_carbon_reduction')))

## Study: Weekends vs Weekdays

In [ ]:
pacer_weekend = pacer_data[pacer_data["IsWeekend"]==True]
pacer_weekday = pacer_data[pacer_data["IsWeekend"]==False]

####  Average Checkouts Per Day

In [ ]:
average_checkouts_weekend = len(pacer_weekend) / (2 * total_days)
average_checkouts_weekday = len(pacer_weekday) / (5 * total_days)

print(f'average_checkouts_weekend: {average_checkouts_weekend:.2f} \naverage_checkouts_weekday: {average_checkouts_weekday:.2f}')

sns.barplot(
    x=['Weekend', 'Weekday'], 
    y=[average_checkouts_weekend, average_checkouts_weekday], 
    estimator=sum, 
    ci=None,
    palette='muted');
plt.title('Average Checkouts Per Day')
plt.savefig(str(os.path.join(os.getcwd(),'other_graphs','checkouts_weekends_v_weekdays')))

### Commutes over 30 Minutes - Weekends vs. Weekdays

In [ ]:
percent_30minplus_weekend = 100 *len(pacer_weekend[pacer_weekend['TripOver30Mins'] == 'Y']) / len(pacer_weekend)
percent_30minplus_weekday = 100 *len(pacer_weekday[pacer_weekday['TripOver30Mins'] == 'Y']) / len(pacer_weekday)

sns.barplot(
    x=['Weekend', 'Weekday'], 
    y=[percent_30minplus_weekend, percent_30minplus_weekday], 
    estimator=sum, 
    ci=None,
    palette='muted');
plt.title('Percent of Checkouts over 30 Minutes')

plt.savefig(str(os.path.join(os.getcwd(),'other_graphs','over_30min_checkouts_weekends_days')))

### Usage Patterns - Weekends vs. Weekdays

In [ ]:
numeric_time_series_wkend = pacer_weekend["CheckoutDateTime"].dt.hour + .01*pacer_weekend["CheckoutDateTime"].dt.minute
numeric_time_series_wkday = pacer_weekday["CheckoutDateTime"].dt.hour + .01*pacer_weekday["CheckoutDateTime"].dt.minute

fig, axes = plt.subplots(1, 2, figsize=(14,5))
# fig.suptitle('1 row x 2 columns axes with no data')

# Weekend
axes[0].set_title('Weekend Checkouts')
weekend_histplot = sns.histplot(data=numeric_time_series_wkend, bins=24, element="step", kde=True, ax=axes[0])
weekend_histplot.set_xlabel("Time of Day", fontsize = 12)

# Weekday
axes[1].set_title('Weekday Checkouts')
weekday_histplot = sns.histplot(data=numeric_time_series_wkday, bins=24, element="step", kde=True, ax=axes[1])
weekday_histplot.set_xlabel("Time of Day", fontsize = 12)

plt.savefig(str(os.path.join(os.getcwd(),'other_graphs',f'weekend_checkout_histogram_{start_date}_{end_date}')))

## Study: Top 5 Busiest Stations

In [ ]:
top_5 = summary_df.sort_values(by="Checkout Count", ascending=False)[0:5].iloc[:, [0, 3]]
top_5['Percent of Total'] = top_5["Checkout Count"]/ summary_df["Checkout Count"].sum() * 100
top_5

## Study: 5 Least Busy Stations

In [ ]:
bottom_5 = summary_df.sort_values(by="Checkout Count", ascending=True)[0:5].iloc[:, [0, 3]]
bottom_5['Percent of Total'] = bottom_5["Checkout Count"]/ summary_df["Checkout Count"].sum() * 100
bottom_5

## Study: Seasons

In [ ]:
# The seasons are saved as numbers...
pacer_data['CheckoutDateTime'].dt.month.unique()

In [ ]:
# separating the data out into smaller 'season' tables
winter_pacer_data = pd.concat(
    [pacer_data[pacer_data['CheckoutDateTime'].dt.month == 12],
     pacer_data[pacer_data['CheckoutDateTime'].dt.month.between(1, 2)]])
spring_pacer_data = pacer_data[pacer_data['CheckoutDateTime'].dt.month.between(3, 5)]
summer_pacer_data = pacer_data[pacer_data['CheckoutDateTime'].dt.month.between(6, 8)]
fall_pacer_data = pacer_data[pacer_data['CheckoutDateTime'].dt.month.between(9, 11)]
# these time series are a condensed way to visualize the time
numeric_time_series_winter = winter_pacer_data["CheckoutDateTime"].dt.hour + .01*winter_pacer_data["CheckoutDateTime"].dt.minute
numeric_time_series_spring = spring_pacer_data["CheckoutDateTime"].dt.hour + .01*spring_pacer_data["CheckoutDateTime"].dt.minute
numeric_time_series_summer = summer_pacer_data["CheckoutDateTime"].dt.hour + .01*summer_pacer_data["CheckoutDateTime"].dt.minute
numeric_time_series_fall = fall_pacer_data["CheckoutDateTime"].dt.hour + .01*fall_pacer_data["CheckoutDateTime"].dt.minute

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16,14))
# Winter
axes[0, 0].set_title('Winter', fontsize = 18)
winter_histplot = sns.histplot(data=numeric_time_series_winter, bins=24, element="step", kde=True, ax=axes[0,0])
winter_histplot.set_xlabel("Time of Day", fontsize = 12)
# Spring
axes[0, 1].set_title('Spring', fontsize = 18)
spring_histplot = sns.histplot(data=numeric_time_series_spring, bins=24, element="step", color='green', kde=True, ax=axes[0,1])
spring_histplot.set_xlabel("Time of Day", fontsize = 12)
# Fall
axes[1, 1].set_title('Fall', fontsize = 18)
fall_histplot = sns.histplot(data=numeric_time_series_fall, bins=24, element="step", kde=True, color='orange', ax=axes[1,1])
fall_histplot.set_xlabel("Time of Day", fontsize = 12)
# Summer
axes[1, 0].set_title('Summer', fontsize = 18)
summer_histplot = sns.histplot(data=numeric_time_series_summer, bins=24, element="step", kde=True, color='r', ax=axes[1,0])
summer_histplot.set_xlabel("Time of Day", fontsize = 12)

plt.savefig(str(os.path.join(os.getcwd(),'other_graphs','seasonal_histograms')))